In [10]:
pip install xgboost

SyntaxError: invalid syntax (1376863068.py, line 1)

In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load data
customers = pd.read_csv('customers.csv')
transactions = pd.read_csv('transactions.csv')
engagements = pd.read_csv('engagements.csv')
marketing = pd.read_csv('marketing.csv')

# Data cleaning
# Convert date columns to datetime
customers['join_date'] = pd.to_datetime(customers['join_date'])
customers['last_purchase_date'] = pd.to_datetime(customers['last_purchase_date'])
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
marketing['campaign_date'] = pd.to_datetime(marketing['campaign_date'])

# Merge datasets
data = customers.merge(transactions, on='customer_id', how='left')
data = data.merge(engagements, on='customer_id', how='left')
data = data.merge(marketing, on='customer_id', how='left')

# Feature engineering
data['customer_lifetime'] = (data['last_purchase_date'] - data['join_date']).dt.days
data['total_spent'] = data.groupby('customer_id')['transaction_amount'].transform('sum')
data['avg_transaction_value'] = data['total_spent'] / data.groupby('customer_id')['transaction_id'].transform('count')
data['transaction_frequency'] = data.groupby('customer_id')['transaction_id'].transform('count')
data['recency'] = (pd.to_datetime('today') - data['last_purchase_date']).dt.days

# Encode categorical variables
categorical_features = ['gender', 'location', 'product_category', 'promotion_type']
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# Select features for modeling
features = ['age', 'customer_lifetime', 'total_spent', 'avg_transaction_value', 
            'transaction_frequency', 'recency', 'number_of_site_visits', 
            'number_of_emails_opened', 'number_of_clicks'] + [col for col in data if col.startswith(tuple(categorical_features))]

X = data[features]
y = data['total_spent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model training and evaluation
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'XGBoost': XGBRegressor()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'{name} Performance:')
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'R²: {r2_score(y_test, y_pred)}')
    print('-'*30)

# Select best model based on evaluation metrics
best_model = XGBRegressor()
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Business impact analysis
# (Add detailed analysis here based on the predicted CLV and its implications for marketing and resource allocation)

# Save the notebook


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <A860C87F-F0F6-32B7-824B-DC2534A91D6C> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/loc